<a href="https://colab.research.google.com/github/sjyogesh23/YoutubeApiDS/blob/main/YoutubeChannel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
#!pip install --upgrade google-api-python-client
!pip install isodate

In [24]:
import pandas as pd
import numpy as np

In [25]:
with open('api.txt', 'r') as file:
    api_key = file.read().strip()
developer_key = api_key

In [26]:
channel_id = ['UCRI00CwLZdLRCWg5BdDOsNw'] #CANADIANLAD

In [27]:
from googleapiclient.discovery import build
from IPython.display import JSON

youtube = build('youtube', 'v3', developerKey=developer_key)

request = youtube.channels().list(
    part="snippet,contentDetails,statistics",
    id=','.join(channel_id)
)
response = request.execute()

#JSON(response)

In [28]:
def get_channel_stats(youtube, channel_id):
  all_data = []
  request = youtube.channels().list(
    part="snippet,contentDetails,statistics",
    id=','.join(channel_id)
  )
  response = request.execute()
  for items in response['items']:
    data = {'channelName': items['snippet']['title'],
            'subscribers': items['statistics']['subscriberCount'],
            'views': items['statistics']['viewCount'],
            'totalVideos': items['statistics']['videoCount'],
            'playlistId': items['contentDetails']['relatedPlaylists']['uploads']
    }
    all_data.append(data)

  return pd.DataFrame(all_data)

get_channel_stats(youtube, channel_id)

,channelName,subscribers,views,totalVideos,playlistId
0,The Canadian Lad,1800000,471097778,315,UURI00CwLZdLRCWg5BdDOsNw


In [29]:
def get_video_ids(youtube, playlist_id, maxResults):
  video_id = []
  playlistId = playlist_id
  request = youtube.playlistItems().list(
    part="snippet,contentDetails",
    playlistId=playlist_id,
    maxResults = maxResults
  )
  response = request.execute()

  video_id.append(item['contentDetails']['videoId'] for item in response['items'])

  next_page_token = response.get('nextPageToken')
  while next_page_token is not None:
    request = youtube.playlistItems().list(
      part="contentDetails",
      playlistId=playlist_id,
      maxResults = maxResults,
      pageToken = next_page_token
    )
    response = request.execute()
    video_id += [item['contentDetails']['videoId'] for item in response['items']]
    next_page_token = response.get('nextPageToken')



  return video_id

video_id = get_video_ids(youtube, get_channel_stats(youtube, channel_id).playlistId[0], 50)
len(video_id)

267

In [30]:
def get_video_details(youtube, video_id):
  video_id = list(map(str, video_id))
  all_video_info = []
  for i in range(0, len(video_id), 50):
    request = youtube.videos().list(
      part="snippet,contentDetails,statistics",
      id=','.join(video_id[i:i+50])
    )
    response = request.execute()

    for video in response['items']:
      stats = {'snippet' : ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
              'statistics' : ['viewCount', 'likeCount', 'favouriteCount', 'commentCount'],
              'contentDetails' : ['duration', 'definition', 'caption']
              }
      video_info = {}
      video_info['video_id'] = video['id']

      for k in stats.keys():
        for v in stats[k]:
          try:
            video_info[v] = video[k][v]
          except:
            video_info[v] = None

      all_video_info.append(video_info)

  return pd.DataFrame(all_video_info)


df1 = get_video_details(youtube, video_id)

#Data pre-processing

In [31]:
df = df1.copy()
df.head(1)

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption
0,4euOBxd9eag,The Canadian Lad,Echo - They Destroyed My Favorite Villain!,Go to https://TryFum.com/CANADIANLAD and use c...,"[Echo, Echo - They Destroyed My Favorite Villa...",2024-01-28T05:23:08Z,207382,14938,None,1725,PT10M38S,hd,false


In [32]:
df.drop(columns=['video_id','channelTitle','favouriteCount'], inplace=True)
df.head(1)

,title,description,tags,publishedAt,viewCount,likeCount,commentCount,duration,definition,caption
0,Echo - They Destroyed My Favorite Villain!,Go to https://TryFum.com/CANADIANLAD and use c...,"[Echo, Echo - They Destroyed My Favorite Villa...",2024-01-28T05:23:08Z,207382,14938,1725,PT10M38S,hd,false


In [33]:
df['publishedAt'] = pd.to_datetime(df['publishedAt'])

df['publishedAt_day'] = df['publishedAt'].dt.day
df['publishedAt_month'] = df['publishedAt'].dt.month
df['publishedAt_year'] = df['publishedAt'].dt.year
df['publishedAt_hr'] = df['publishedAt'].dt.hour
df['publishedAt_mins'] = df['publishedAt'].dt.minute
df['publishedAt_weekday'] = df['publishedAt'].dt.day_name()

cols = list(df.columns)
new_cols = ['publishedAt_day', 'publishedAt_month', 'publishedAt_year',
            'publishedAt_hr', 'publishedAt_mins','publishedAt_weekday']
for col in new_cols:
    cols.remove(col)
for i, col in enumerate(new_cols):
    cols.insert(3 + i, col)
df = df[cols]
df = df.drop(columns=['publishedAt'])

In [34]:
df.head(1)

,title,description,tags,publishedAt_day,publishedAt_month,publishedAt_year,publishedAt_hr,publishedAt_mins,publishedAt_weekday,viewCount,likeCount,commentCount,duration,definition,caption
0,Echo - They Destroyed My Favorite Villain!,Go to https://TryFum.com/CANADIANLAD and use c...,"[Echo, Echo - They Destroyed My Favorite Villa...",28,1,2024,5,23,Sunday,207382,14938,1725,PT10M38S,hd,false


In [35]:
import isodate
df['durationsecs'] = df['duration'].apply(lambda x: isodate.parse_duration(x))
df['durationsecs'] = df['durationsecs'].astype('timedelta64[s]').astype(int)

df['duration_days'] = df['durationsecs'] // (24 * 3600)
df['duration_hrs'] = (df['durationsecs'] % (24 * 3600)) // 3600
df['duration_secs'] = df['durationsecs'] % 60

df['duration_mins'] = (((df['durationsecs'] % 3600) // 60) + (df['duration_secs'] / 60)).round(2)

new_cols = ['duration_days', 'duration_hrs', 'duration_mins', 'duration_secs']
cols = list(df.columns)
for col in new_cols:
    cols.remove(col)
for i, col in enumerate(new_cols):
    cols.insert(12 + i, col)

df = df[cols]

In [36]:
df.head(1)

,title,description,tags,publishedAt_day,publishedAt_month,publishedAt_year,publishedAt_hr,publishedAt_mins,publishedAt_weekday,viewCount,likeCount,commentCount,duration_days,duration_hrs,duration_mins,duration_secs,duration,definition,caption,durationsecs
0,Echo - They Destroyed My Favorite Villain!,Go to https://TryFum.com/CANADIANLAD and use c...,"[Echo, Echo - They Destroyed My Favorite Villa...",28,1,2024,5,23,Sunday,207382,14938,1725,0,0,10.63,38,PT10M38S,hd,false,638


In [37]:
df = df.drop(columns=['duration', 'duration_days', 'duration_hrs', 'durationsecs', 'duration_secs'])
df.head(1)

,title,description,tags,publishedAt_day,publishedAt_month,publishedAt_year,publishedAt_hr,publishedAt_mins,publishedAt_weekday,viewCount,likeCount,commentCount,duration_mins,definition,caption
0,Echo - They Destroyed My Favorite Villain!,Go to https://TryFum.com/CANADIANLAD and use c...,"[Echo, Echo - They Destroyed My Favorite Villa...",28,1,2024,5,23,Sunday,207382,14938,1725,10.63,hd,false


In [38]:
df.dtypes

,0
title,object
description,object
tags,object
publishedAt_day,int32
publishedAt_month,int32
publishedAt_year,int32
publishedAt_hr,int32
publishedAt_mins,int32
publishedAt_weekday,object
viewCount,object


In [39]:
for col in ['viewCount', 'likeCount', 'commentCount']:
  df[col] = pd.to_numeric(df[col], errors="coerce")

df['caption'] = df['caption'].apply(lambda x: 1 if x == 'True' else 0)
df['definition'] = df['definition'].apply(lambda x: 1 if x == 'hd' else 0)
df.dtypes

,0
title,object
description,object
tags,object
publishedAt_day,int32
publishedAt_month,int32
publishedAt_year,int32
publishedAt_hr,int32
publishedAt_mins,int32
publishedAt_weekday,object
viewCount,int64


In [40]:
df['tagcount'] = df['tags'].apply(lambda x: 0 if x is None else len(x))
cols = list(df.columns)
cols.remove('tagcount')
cols.insert(3, 'tagcount')
df = df[cols]
df.head(1)

,title,description,tags,tagcount,publishedAt_day,publishedAt_month,publishedAt_year,publishedAt_hr,publishedAt_mins,publishedAt_weekday,viewCount,likeCount,commentCount,duration_mins,definition,caption
0,Echo - They Destroyed My Favorite Villain!,Go to https://TryFum.com/CANADIANLAD and use c...,"[Echo, Echo - They Destroyed My Favorite Villa...",17,28,1,2024,5,23,Sunday,207382,14938,1725,10.63,1,0


In [41]:
df['tags'] = df['tags'].fillna('')

#Data Analysis